In [ ]:
import cv2 
import numpy as np
import os
from matplotlib import pyplot as plt
from func_hair import smplextract_hair
from PIL import Image
from sklearn.metrics import jaccard_score
from skimage.feature import graycomatrix, graycoprops
import pyfeats
from pathlib import Path
import math

pathFolder_train='D:/Respaldo/Documents/MAIA/UNICAS/AImageP/project/FILES/Dataset/train/' 
train_Array = [x for x in os.listdir(pathFolder_train) if os.path.isfile(os.path.join(pathFolder_train,x))]
pathFolder_test='D:/Respaldo/Documents/MAIA/UNICAS/AImageP/project/FILES/Dataset/test/' 
test_Array = [x for x in os.listdir(pathFolder_test) if os.path.isfile(os.path.join(pathFolder_test,x))]
pathFolder_GTtrain='D:/Respaldo/Documents/MAIA/UNICAS/AImageP/project/FILES/Dataset/GT_train/'
GTtrain_Array = [x for x in os.listdir(pathFolder_GTtrain) if os.path.isfile(os.path.join(pathFolder_GTtrain,x))]
outputFolder_path="D:/Respaldo/Documents/MAIA/UNICAS/AImageP/project/FILES/Dataset/Seg_train"
pathFolder_whHair_train="D:/Respaldo/Documents/MAIA/UNICAS/AImageP/project/FILES/Dataset/whHair_train/"
pathFolder_masks_train="D:/Respaldo/Documents/MAIA/UNICAS/AImageP/project/FILES/Dataset/seg_res_train/"
pathFolder_gtres_train="D:/Respaldo/Documents/MAIA/UNICAS/AImageP/project/FILES/Dataset/gt_res_train/"

#
def remove_fov(seg_im):

    seg_im=cv2.cvtColor(seg_im,cv2.COLOR_RGB2GRAY)
    ret,thresh = cv2.threshold(seg_im,127,255,0)
    x,y = (find_center(seg_im))

# find contours in the binary image
    contours,hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    Distances = []
    for c in contours:
   # calculate moments for each contour
        M = cv2.moments(c)
   # calculate x,y coordinate of center
        cX = int(M["m10"] / (M["m00"] + 0.001))
        cY = int(M["m01"] / (M["m00"]+ 0.001))
        Distances.append(math.dist([x, y], [cX, cY]))
   ##print(cX,cY)

    #print(Distances)
    index_min = np.argmin(Distances, axis=0)
    cnt = contours[index_min]

    blank = np.zeros(thresh.shape, dtype='uint8')
 
    cv2.drawContours(blank, [cnt], -1, (255),cv2.FILLED)
    

    M = cv2.moments(cnt)
    # calculate x,y coordinate of center
    c_x = int(M["m10"] / (M["m00"] + 0.001))
    c_y = int(M["m01"] / (M["m00"]+ 0.001))
    #print(c_x,c_y)
    #cv2.imshow("Image", blank)
    #cv2.waitKey(500)

    return blank

def find_center(image):
    (h, w) = image.shape[:2] #w:image-width and h:image-height
    (cx,cy)=(w//2, h//2)
    return cx,cy

def find_center_im(image):
    (h, w) = image.shape[:2] #w:image-width and h:image-height
    (cx,cy)=cv2.circle(image, (w//2, h//2), 7, (255, 255, 255), -1) 
    cv2.putText(image, "center", (w//2- 20, h//2 - 20),
	cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
	# show the image
    #cv2.imshow("Image", image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cv2.waitKey(500)
    plt.imshow(image, cmap='gray')
    plt.show()
    return cx,cy

def crop_center(img,cropx,cropy):
    y,x,c = img.shape
    x
    y
    c
    # startx = x//2 - cropx//2 round(cropx*0.75)
    # starty = y//2 - cropy//2 
    x1 = 200 
    x2 = 900 
    y1= 50
    y2 = 750
  
    return img[y1:y2,x1:x2,:]
    #img[starty:starty+cropy, startx:startx+cropx, :]

def create_patch(imag1):

    (h, w) = imag1.shape[:2]
    x_coord_center= w//2 #imag1.shape[1]
    y_coord_center= h//2 #imag1.shape[0]
    patch_size = 40
    image_patch=crop_center(imag1,x_coord_center, y_coord_center)
    cv2.putText(imag1, "center", (w//2- 20, h//2 - 20),
	  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    cv2.imshow("Image", imag1)
    cv2.waitKey(500)
   
    return image_patch
 
def substract_2_im(im1,im2):
    prep_img = im1 * im2
    return prep_img

def invert_image(imagem):
    imagem = (255-imagem)
    return imagem

def segmentation(img_o):
   
    down_width = 817
    down_height = 613
    down_points = (down_width, down_height)
    
    img_r= extract_hair(resized_down,256,192)
    img_r=img_o
  

    #apply median filter to remove noise 
    median = cv2.medianBlur(img_r,5) 
    #cv2.imshow("mediana",median)
    Z = median.reshape((-1,3))
   
   ############################################################################
   ##############k-means#######################################################
   #########################################################################
   
    #    convert to np.float32
    Z = np.float32(Z)
    # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 8 #default number for optimal results
    flags=cv2.KMEANS_RANDOM_CENTERS # select initial cluster based on randomness
    ret,label,center=cv2.kmeans(Z,K,None,criteria,10,flags)

    # Now convert back into uint8, and make original image
    center = np.uint8(center)
    res = center[label.flatten()]
    kmeans_img = res.reshape((img_r.shape))

   
    # =============================================================================
    # =============================================================================
    #    Adaptive histogram equalization  
    # =============================================================================
    clahe = cv2.createCLAHE(clipLimit=3., tileGridSize=(8,8))

    hsv = cv2.cvtColor(kmeans_img, cv2.COLOR_BGR2HSV)# convert from BGR to HSV color space
     
    h, s, v = cv2.split(hsv)  # split on 3 different channels
    #   apply CLAHE to the L-channel
    h1 = clahe.apply(h)
    s1 = clahe.apply(s)
    v1 = clahe.apply(v)

    lab = cv2.merge((h1,s1,v1))  # merge channels
    #cv2.imshow("lab",lab)
  
        
    Enhance_img= cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)  # convert from LAB to BGR
    
  
        
    # =============================================================================
    #    making the mask for grabcut
    # =============================================================================
    
    
    #Green color was extracted from the image and applying thresholding mask was obtained. 
    hsv = cv2.cvtColor(Enhance_img, cv2.COLOR_BGR2HSV)  
    # function returns an array of elements equal to 0 if the elements of the given array do not lie between the two arrays representing the upper bounds and the lower bounds.
    #where upper lower bounds are values of hsv   
    lower_green = np.array([50,100,100])
    upper_green = np.array([100,255,255])
    mask_g = cv2.inRange(hsv, lower_green, upper_green)
   
    # The method returns two outputs. The first is the threshold that was used and the second output is the thresholded image
    ret,inv_mask = cv2.threshold(mask_g,127,255,cv2.THRESH_BINARY_INV)
   
    #  
    res = cv2.bitwise_and(img_r,img_r, mask= mask_g)
    #mask haivng zeros 
    mask = np.zeros(img_r.shape[:2],np.uint8)
    # Create 2 arrays for background and foreground model 
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
      
    
    
  #####################################################################################
    # =============================================================================
    #      GRABCUT
    # =============================================================================
    #number sum decreased it showed better segmentation to avoid inversion we set lower number
    #we used 70% of pixel value 
    if (np.sum(inv_mask[:])<8507940):
        newmask = inv_mask
    #exceeds threshold we use inverted masks 

    
        # wherever it is marked white (sure foreground), change mask=1
        # wherever it is marked black (sure background), change mask=0
        mask[newmask == 0] = 0
        mask[newmask == 255] = 1
        #change pixel 0,2 to background and 1,3 to foreground 
        mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
        GrabCut_img2 = img_r*mask2[:,:,np.newaxis]
       
    else:
         #initializing the Rectangle based on the image dimention
        s = (img_r.shape[0] / 10, img_r.shape[1] / 10)
        rect = (int(s[0]), int(s[1]), int(img_r.shape[0]) -int((3/10)) * int(s[0]), int(img_r.shape[1]) - int(s[1]))
        #applying grab cut applies rectangle around the image
        cv2.grabCut(lab,mask,rect,bgdModel,fgdModel,10,cv2.GC_INIT_WITH_RECT)
        mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
        GrabCut_img2= img_r*mask2[:,:,np.newaxis]
      
     

    # =============================================================================
    # Binarization
    # =============================================================================
    imgmask2 = cv2.medianBlur(GrabCut_img2,5)
    ret,Segmented_mask2 = cv2.threshold(imgmask2,0,255,cv2.THRESH_BINARY)
    cv2.waitKey(500)
    plt.imshow(GrabCut_img2)
    return Segmented_mask2
  # ==================================================
av_Jaccard = 0
for i in range(0,200):#range(len(train_Array)): 
    wjpg=train_Array[i]
    output= wjpg.split(".", 1)
    id=output[0]
    img_o=cv2.imread("D:/Respaldo/Documents/MAIA/UNICAS/AImageP/project/FILES/Dataset/200Jaccard/"+id +'_wh.jpg') #PREPROCES WHAIR
    #img_r=cv2.imread(pathFolder_masks_train+id+'_segmask.png')
    #print((train_Array[i]))
    print(id)
    img_seg = segmentation(img_o)

    ####################fos performed################################################################
    #print("shapeimgdseg",img_seg.shape)
    # cv2.imwrite(os.path.join(outputFolder_path, train_Array[i]+'_segmask.png'),img_seg)
    # img_seg[img_seg == 255] = 1
    # mfos=img_o[img_seg]
    # print(i)
    # print(type(mfos))
    # print(type(img_seg))
    # print("shapemfos",mfos.shape)
    # cv2.imshow("seg",img_seg)
    # cv2.waitKey(0)
    #img_rfv=remove_fov(img_seg)
    #print('s',img_rfv.shape)
    #img_rfv=cv2.cvtColor(img_rfv,cv2.COLOR_GRAY2RGB)
    #print('ns',img_rfv.shape)
    #cv2.imwrite(os.path.join(outputFolder_path,id+'_segmask.png'),img_seg)
    #print("immseg",img_rfv)
    cv2.imshow("seg mask",img_seg)
    h1,w1,c1=img_seg.shape
    if img_seg[int(h1/2),int(w1/2)].all()==0:
       img_seg=255-img_seg
    #cv2.imshow("final mask",img_rfv)
    #cv2.waitKey(500)
    #img_seg[img_seg == 255] = 1
    #img_rfv[img_rfv==255]=1
    #substraction=substract_2_im(img_o,img_seg)
    #substraction=substract_2_im(img_o,img_rfv)
    #print("maskwimag",substraction)
    #cv2.imshow("subs",substraction)
    #cv2.waitKey(0)
    #extraction=create_patch(img_o)
    #cv2.imshow("ext",extraction)
    #cv2.waitKey(0)
    img_gt=cv2.imread(pathFolder_GTtrain+GTtrain_Array[i])
    cv2.imshow("gt",img_gt)
    #cv2.destroyAllWindows()
   
    resized_up=cv2.resize(img_seg, (img_gt.shape[1], img_gt.shape[0]))
    print(resized_up.shape)
    cv2.imshow("res up",resized_up)
    cv2.waitKey(100)
    cv2.destroyAllWindows()
    gt = (img_gt/255).astype(np.uint8)
    seg = (img_seg/255).astype(np.uint8)
    ################################jaccard###############################
    # Jaccard :
    J = jaccard_score(gt.ravel(), seg.ravel())
   # J = jaccard_score(gt.flatten(), seg.flatten())
    print(J)
    av_Jaccard = av_Jaccard + J
    print(av_Jaccard)

final_Jacc = av_Jaccard/200 #len(train_Array) #/4
print(final_Jacc)

# #jaccard_score(gt,resized_up)



ModuleNotFoundError: ignored

# **Hair removal function**

In [ ]:
#Hair removal function

import cv2 as cv
import numpy as np
import os
from skimage.transform import resize


#function that performs the sum of the structure elements in different directions
def sum_operation_se_different_directions(img,operation, width,height,n_se):
    # create SEs
    base = np.zeros([width, width])
    k = int(width / 2 - height / 2)
    while k <= (width / 2 + height / 2):
        base = cv.line(base, (0, k), (width, k), 255) #drawing a line
        k = k + 1
        #print(k)
    SEs = []
    SEs.append(base)
    angle = 180.0 / n_se
    #Aplying affine transformations to the elements 
    for k in range(1, n_se):
        SEs.append(cv.warpAffine(base, cv.getRotationMatrix2D((base.shape[0] / 2, base.shape[1] / 2), k * angle, 1.0),(width, width)))
    # cv.imshow("see",SEs[0])
    #print(SEs[2].shape)
    #Performing the sum of the elements with a morphology operation, in this case blackhats
    open_sum = np.uint16(0*cv.morphologyEx(img, operation, np.uint8(SEs[0])))
    for se in SEs:
        open_sum += cv.morphologyEx(img, operation, np.uint8(se))
    #Applying the normalization
    result= cv.normalize(open_sum, 0, 255, norm_type=cv.NORM_MINMAX)
    return np.uint8(result)


def smplextract_hair(src):
    
    # Convert the original image to grayscale
    img_gray=cv.cvtColor(src,cv.COLOR_BGR2GRAY)
    #Applying the contrast enhance to the image, setting a contrast limit to avoid noise amplification
    clahe = cv.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))
    contrast_enhanced_gray_img = clahe.apply(img_gray)

    #Selecting the size and number of structure elements
    width= 29
    height=2
    n_se=15
    
    #cv.imshow("original_image",src)
    #Performing the sum of black hats in order to obtain the contours of the hairs with respect to the direction
    sum_black_hats=sum_operation_se_different_directions(contrast_enhanced_gray_img,cv.MORPH_BLACKHAT, width,height,n_se)
    #cv.imshow("sum_blackhats", sum_black_hats)
    #Applying the threshold to increase the pixel intensities in the mask
    ret, bin_img = cv.threshold(sum_black_hats, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    #cv.imshow("Thresholded Mask",bin_img)
    #Applying dilation to prepare the mask for the next step
    dilated_bin_img=cv.dilate(bin_img,cv.getStructuringElement(cv.MORPH_ELLIPSE,(3,3)))
    #cv.imshow("dilated", dilated_bin_img)
    # Inpaint the original image depending on the mask
    dst = cv.inpaint(src,dilated_bin_img,7,cv.INPAINT_TELEA)
    #cv.imshow("Inpaint",dst)
    #cv.waitKey(50)
    return dst